# Introduction to std::string
C++ has introduced two additional string types (**WHICH ARE DIFFERENT FROM C-STYLED STRINGS**) into the language that are much easier and safer to work with: 
```
- std::string 
- std::string_view (C++17). 
```

Although std::string and std::string_view aren’t fundamental types, they’re straightforward and useful enough that we’ll introduce them here.

In [1]:
#include <iostream>
#include <string> // allows use of std::string

std::string name { "Alex" }; // initialize name with string literal "Alex"
// The use of braces ({}) for initialization is a feature introduced in C++11, known as uniform initialization.
std::cout << "My name is: " << name << '\n';
name = "John";               // change name to "John"
std::cout << "My name is: " << name << '\n';


My name is: Alex
My name is: John


### Uniform Initialization
 - The use of braces ({}) for initialization is a feature introduced in C++11, known as uniform initialization.
 - The use of braces for initialization (int x { 5 };) is recommended in modern C++ because it provides more consistent and less error-prone initialization syntax compared to traditional methods. It also helps prevent narrowing conversions and allows the use of the auto keyword for type inference. 

### std::string can handle strings of different lengths
One of the neatest things that std::string can do is hold strings of different sizes:

In [2]:
std::string name { "Alex" }; // initialize name with string literal "Alex"
std::cout << name << '\n';

name = "Jason";              // change name to a longer string
std::cout << name << '\n';

name = "Jay";                // change name to a shorter string
std::cout << name << '\n';

Alex
Jason
Jay


In the above example, name is initialized with the string "Alex", which contains five characters (four explicit characters and a null-terminator). We then set name to a larger string, and then a smaller string. std::string has no problem handling this!

### String input with std::cin
```
std::cout << "Enter your full name: ";
std::string name{};
std::cin >> name; // this won't work as expected since std::cin breaks on whitespace
```

### Use std::getline() to input text

```
std::cout << "Enter your full name: ";
std::string name{};
std::getline(std::cin >> std::ws, name); // read a full line of text into name
```

### What the heck is std::ws? (In above line)
C++ has output manipulators, which allow us to alter the way output is displayed. 

C++ also supports input manipulators, which alter the way that input is accepted. The std::ws input manipulator tells std::cin to ignore any leading whitespace before extraction. Leading whitespace is any whitespace character (spaces, tabs, newlines) that occur at the start of the string.

#### Why is std::ws needed?
When you enter a value using operator>>, std::cin not only captures the value, it also captures the newline character ('\n') that occurs when you hit the enter key. So when we type 2 and then hit enter, std::cin captures the string "2\n" as input. It then extracts the value 2 to variable choice, leaving the newline character behind for later. Then, when std::getline() goes to extract text to name, it sees "\n" is already waiting in std::cin, and figures we must have previously entered an empty string! Definitely not what was intended.

In [3]:
    std::cout << "Pick 1 or 2: ";
    int choice{};
    std::cin >> choice;

    std::cout << "Now enter your name: ";
    std::string name{};
    std::getline(std::cin >> std::ws, name); // note: added std::ws here, if not then \n would be your name as explained earlier

    std::cout << "Hello, " << name << ", you picked " << choice << '\n';

    return 0;

Pick 1 or 2: Now enter your name: Hello, rohirto, you picked 2


#### Best practice

If using std::getline() to read strings, use std::cin >> std::ws input manipulator to ignore leading whitespace. This needs to be done for each std::getline() call, as std::ws is not preserved across calls.

### The length of a std::string

In [4]:
std::string name{ "Alex" };
std::cout << name << " has " << name.length() << " characters\n";

Alex has 4 characters


The length() function isn’t a normal standalone function -- it’s a special type of function that is nested within std::string called a member function. Because the length() member function is declared inside of std::string, it is sometimes written as 
```std::string::length()``` in documentation.

Also note that std::string::length() returns an unsigned integral value (most likely of type size_t). If you want to assign the length to an int variable, you should static_cast it to avoid compiler warnings about signed/unsigned conversions:
```
int length { static_cast<int>(name.length()) };
```

In C++20, you can also use the ```std::ssize()``` function to get the length of a std::string as a signed integral value

#### Initializing a std::string is expensive

Whenever a std::string is initialized, a copy of the string used to initialize it is made. Making copies of strings is expensive, so care should be taken to minimize the number of copies made.

#### Do not pass std::string by value

When a std::string is passed to a function by value, the std::string function parameter must be instantiated and initialized with the argument. This results in an expensive copy. We’ll discuss what to do instead (use std::string_view) 


#### Returning a std::string

However, as a rule of thumb, it is okay to return a std::string by value when the expression of the return statement resolves to any of the following:

- A local variable of type std::string.
- A std::string that has been returned by value from a function call or operator.
- A std::string that is created as part of the return statement.

If returning a C-style string literal, use a std::string_view return type instead

#### Literals for std::string
We can create string literals with type std::string by using a s suffix after the double-quoted string literal. (Thus such literals are not c-styled strings)

In [5]:
using namespace std::string_literals; // easy access to the s suffix

std::cout << "foo\n";   // no suffix is a C-style string literal
std::cout << "goo\n"s;  // s suffix is a std::string literal

foo
goo


@0x7f1102661de0

#### Constexpr strings
constexpr std::string isn’t supported at all in C++17 or earlier, and only works in very limited cases in C++20/23. If you need constexpr strings, use std::string_view instead

### But what is consexpr string??
```
constexpr const char* hello = "Hello, World!";
```
A string of Constant characters.

the constexpr specifier is used to declare a compile-time string named hello. The variable is then used in the main function, and it will be evaluated at compile-time.

## Introduction to std::string_view
Unlike fundamental types, initializing and copying a std::string is slow.

To address the issue with std::string being expensive to initialize (or copy), C++17 introduced std::string_view (which lives in the ```<string_view>``` header). std::string_view provides read-only access to an existing string (a C-style string, a std::string, or another std::string_view) without making a copy. Read-only means that we can access and use the value being viewed, but we can not modify it.


In [6]:
#include <string_view>

// str provides read-only access to whatever argument is passed in
void printSV(std::string_view str) // now a std::string_view
{
    std::cout << str << '\n';
}

std::string_view s{ "Hello, world!" }; // now a std::string_view
printSV(s);

Hello, world!


When we initialize std::string_view s with C-style string literal "Hello, world!", s provides read-only access to “Hello, world!” without making a copy of the string. When we pass s to printSV(), parameter str is initialized from s. This allows us to access “Hello, world!” through str, again without making a copy of the string.

Prefer std::string_view over std::string when you need a read-only string, especially for function parameters.

#### std::string_view can be initialized with many different types of strings

A std::string_view object can be initialized with a C-style string, a std::string, or another std::string_view:

In [7]:
std::string_view s1 { "Hello, world!" }; // initialize with C-style string literal
std::cout << s1 << '\n';

std::string s{ "Hello, world!" };
std::string_view s2 { s };  // initialize with std::string
std::cout << s2 << '\n';

std::string_view s3 { s2 }; // initialize with std::string_view
std::cout << s3 << '\n';

Hello, world!
Hello, world!
Hello, world!


#### std::string_view parameters will accept many different types of string arguments

 a std::string_view parameter will accept arguments of type C-style string, a std::string, or std::string_view

In [8]:
printSV("Hello, world!"); // call with C-style string literal

std::string s2{ "Hello, world!" };
printSV(s2); // call with std::string

std::string_view s3 { s2 };
printSV(s3); // call with std::string_view

Hello, world!
Hello, world!
Hello, world!


#### std::string_view will not implicitly convert to std::string

Because std::string makes a copy of its initializer (which is expensive), C++ won’t allow implicit conversion of a std::string_view to a std::string. This is to prevent accidentally passing a std::string_view argument to a std::string parameter, and inadvertently making an expensive copy where such a copy may not be required.

However, if this is desired, we have two options:

- Explicitly create a std::string with a std::string_view initializer (which is allowed, since this will rarely be done unintentionally)
- Convert an existing std::string_view to a std::string using static_cast

The following example shows both options:

In [9]:
void printString(std::string str)
{
	std::cout << str << '\n';
}

std::string_view sv{ "Hello, world!" };
//printString(sv);   // compile error: won't implicitly convert std::string_view to a std::string

std::string s{ sv }; // okay: we can create std::string using std::string_view initializer
printString(s);      // and call the function with the std::string

printString(static_cast<std::string>(sv)); // okay: we can explicitly cast a std::string_view to a std::string

Hello, world!
Hello, world!


#### Assignment changes what the std::string_view is viewing
Assigning a new string to a std::string_view causes the std::string_view to view the new string. It does not modify the prior string being viewed in any way.

#### Literals for std::string_view
Double-quoted string literals are C-style string literals by default. We can create string literals with type std::string_view by using a sv suffix after the double-quoted string literal.



In [10]:
using namespace std::string_literals;      // access the s suffix
using namespace std::string_view_literals; // access the sv suffix

std::cout << "foo\n";   // no suffix is a C-style string literal
std::cout << "goo\n"s;  // s suffix is a std::string literal
std::cout << "moo\n"sv; // sv suffix is a std::string_view literal

foo
goo
moo


@0x7f1102661de0

#### constexpr std::string_view

Unlike std::string, std::string_view has full support for constexpr
This makes constexpr std::string_view the preferred choice when string symbolic constants are needed.

In [11]:
constexpr std::string_view s{ "Hello, world!" }; // s is a string symbolic constant
std::cout << s << '\n'; // s will be replaced with "Hello, world!" at compile-time

Hello, world!


@0x7f1102661de0

#### std::string vs std::string_view
 - Ownership can be expensive. As an owner, it is your responsibility to acquire, manage, and properly dispose of the objects you own. std::string is an owner
 - Viewing is inexpensive. As a viewer, you have no responsibility for the objects you are viewing, but you also have no control over those objects. std::string_view is a viewer


#### std::string_view is best used as a read-only function parameter
This allows us to pass in a C-style string, std::string, or std::string_view argument without making a copy, as the std::string_view will create a view to the argument.

#### Should I prefer std::string_view or const std::string& function parameters?” Advanced

Prefer std::string_view in most cases

#### Modifying a std::string invalidates all views into that std::string.



In [12]:
std::string s { "Hello, world!" };
std::string_view sv { s }; // sv is now viewing s

s = "Hello, universe!";    // modifies s, which invalidates sv (s is still valid)
std::cout << sv << '\n';   // undefined behavior

return 0;

       orld!


#### Revalidating an invalid std::string_view
Invalidated objects can often be revalidated (made valid again) by setting them back to a known good state. For an invalidated std::string_view, we can do this by assigning the invalidated std::string_view object a valid string to view.

In [13]:
std::string s { "Hello, world!" };
std::string_view sv { s }; // sv is now viewing s

s = "Hello, universe!";    // modifies s, which invalidates sv (s is still valid)
std::cout << sv << '\n';   // undefined behavior

sv = s;                    // revalidate sv: sv is now viewing s again
std::cout << sv << '\n';   // prints "Hello, universe!"

       orld!
Hello, universe!


@0x7f1102661de0

#### Be careful returning a std::string_view
std::string_view can be used as the return value of a function. However, this is often dangerous.

Because local variables are destroyed at the end of the function, returning a std::string_view to a local variable will result in the returned std::string_view being invalid, and further use of that std::string_view will result in undefined behavior. For example:
```
std::string_view getBoolName(bool b)
{
    std::string t { "true" };  // local variable
    std::string f { "false" }; // local variable

    if (b)
        return t;  // return a std::string_view viewing t

    return f; // return a std::string_view viewing f
} // t and f are destroyed at the end of the function

std::cout << getBoolName(true) << ' ' << getBoolName(false) << '\n'; // undefined behavior
```

There are two main cases where a std::string_view can be returned safely. First, because C-style string literals exist for the entire program, it’s fine to return C-style string literals from a function that has a return type of std::string_view.

```
std::string_view getBoolName_C(bool b)
{
    if (b)
        return "true";  // return a std::string_view viewing "true"

    return "false"; // return a std::string_view viewing "false"
} // "true" and "false" are not destroyed at the end of the function

std::cout << getBoolName_C(true) << ' ' << getBoolName_C(false) << '\n'; // ok
```

Second, it is generally okay to return a function parameter of type std::string_view:
```
std::string_view firstAlphabetical(std::string_view s1, std::string_view s2)
{
    if (s1 < s2)
        return s1;
    return s2;
}

int main()
{
    std::string a { "World" };
    std::string b { "Hello" };

    std::cout << firstAlphabetical(a, b) << '\n'; // prints "Hello"

    return 0;
}
```

#### View modification functions

Because std::string_view is a view, it contains functions that let us modify our view by “closing the curtains”. This does not modify the string being viewed in any way, just the view itself.

- The remove_prefix() member function removes characters from the left side of the view.
- The remove_suffix() member function removes characters from the right side of the view.




In [14]:
std::string_view st{ "Peach" };
std::cout << st << '\n';

// Remove 1 character from the left side of the view
st.remove_prefix(1);
std::cout << st << '\n';

// Remove 2 characters from the right side of the view
st.remove_suffix(2);
std::cout << st << '\n';

st = "Peach"; // reset the view
std::cout << st << '\n';


Peach
each
ea
Peach


#### std::string_view can view a substring
#### std::string_view may or may not be null-terminated
- A C-style string literal and a std::string are always null-terminated.
- A std::string_view may or may not be null-terminated.
- In almost all cases, this doesn’t matter -- a std::string_view keeps track of the length of the string or substring it is viewing, so it doesn’t need the null-terminator. Converting a std::string_view to a std::string will work regardless of whether or not the std::string_view is null-terminated.